In [29]:
import numpy as np
import cv2 as cv
import os
import shutil
from matplotlib import pyplot as plt
from os import listdir
from os.path import isfile, join

In [30]:
#folder = "../../Documents/ZED2"
folder = "Dataset/Negro"
carpeta = "Dataset/Negro_cropped"
if os.path.exists(carpeta):
    shutil.rmtree(carpeta)
os.makedirs(carpeta)

images = []
for filename in os.listdir(folder):
    img = cv.imread(os.path.join(folder,filename), cv.IMREAD_COLOR)
    if img is not None:
        height, width = img.shape[:2]
        cropped = img[200:height-200, 200:width-200]
        images.append(cropped)
    
plt.imshow(images[0], cmap="gray")


PermissionError: [Errno 13] Permission denied: '/Dataset'

In [ ]:
def img2Binary(img):
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(img_gray,(5,5),0)
    ret,th = cv.threshold(blur,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
    
    return th
    

In [ ]:
def morphOp(th):
    kernel = np.ones((5,5),np.uint8)

    # Dilation
    dilation = cv.dilate(th, kernel, iterations = 20)

    # Erosion
    erosion = cv.erode(dilation, kernel, iterations = 20)

    # Closing -> Erasing the holes inside the object
    closing = cv.morphologyEx(th, cv.MORPH_CLOSE, kernel, iterations = 18)

    # Opening -> Erasing the tails
    kernel = np.ones((13,13),np.uint8)

    opening = cv.morphologyEx(closing, cv.MORPH_OPEN, kernel, iterations = 18)
    #plt.imshow(opening, cmap="gray")
    return opening


In [ ]:
def euclideanDistance(p1, p2):
    # p1, p2 as tuples
    dis = ((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2) ** 0.5
    return dis

In [ ]:
## Finding the contour closest to the center of the image
def getCenteredContour(contours, image_center):
    minDistance = float("inf") ## Highest number 

    for cnt in contours:
        # Find the center of each contour
        M = cv.moments(cnt)
        centerX = int(M["m10"] / M["m00"])
        centerY = int(M["m01"] / M["m00"])
        contour_center = (centerX, centerY)

        # Then calculate the distance between the center of the contour and the center of the image
        distance = euclideanDistance(image_center, contour_center)
        if(distance < minDistance):
            minDistance = distance
            minDistanceContour = cnt
    return minDistanceContour

In [ ]:
## Finding the biggest contour
def getBiggestContour(contours, img):
    biggestArea = 0
    for cnt in contours:
        area = cv.contourArea(cnt)
        if biggestArea < area:
            biggestArea = area
            biggestContour = cnt
    return biggestContour, biggestArea

In [ ]:

def ROI(img_rgb, opening, i):
    copy = img_rgb.copy()
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    contours, hierarchy = cv.findContours(opening, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)

    ## Finding the center of the image
    moment = cv.moments(img_gray)
    x = int(moment ["m10"] / moment["m00"])
    y = int(moment ["m01"] / moment["m00"])
    image_center = (x,y)
    cv.circle(copy, image_center, 3, (255, 100, 0), 2)


    mostCenteredContour = getCenteredContour(contours, image_center)
    cv.drawContours(copy, mostCenteredContour, -1, (255,0,0), 3)

    x,y,w,h = cv.boundingRect(mostCenteredContour)
    cv.rectangle(copy,(x,y),(x+w,y+h),(255,0,0),8)

    dst = cv.Canny(img_gray, 0, 150)
    blured = cv.blur(dst, (5,5), 0)

    [X, Y, W, H] = cv.boundingRect(mostCenteredContour)
    cropped = img_rgb[Y:Y+H, X:X+W]
    
    cv.imwrite(os.path.join(carpeta, str(i)+".jpg"), cropped)

A FUNCIONAR

In [ ]:
i=0
for img in images:
    th = img2Binary(img)
    opening = morphOp(th)
    ROI(img, opening, i)
    i += 1